# Цели проекта
Основной целью проекта является демонстрация освоения инструментария machine learning для решения прикладной задачи.

# Задача проекта
Задачей проекта явяляется созданией модели, работающей на нейросетях, для определения гендерных предпочтений по фотографии.
В данном решении проводится только анализ мужских фотографий лиц, без привязки к возрасту/географии/предпочтениям конкретных людей.
Итоговая нейросеть должна по фотографии определить, какой пол более привлекателей для обладателя фотографии - мужской или женский.
Проект не призван вдаваться в полемику о природе гендерных предпочтений, а явялется простым аппроксиматором собранных данных.

# Информация о данных
Данные были собраны путем парсинга профилей на tinder.com. 
Был создан профиль мужского пола, который ищет мужчину, написан скрипт, имитирующий активность реального человека и сохраняеющий первые фотографии профилей.
Для контрольной группы предпочтения профиля были изменены на женщин и собран аналогичный по размеру датасет. 
Датасет содерджит 8657 изображение размера 512х512 пикселей для контрольной группы и 7564 аналогичных изображений для исследуемой группы.
Tinder закрыл свой API после аналогичного случая парсинга во Флориде, когда были скачены профили больше, чем 40000 пользователей, однако все собранные фотографии находятся в открытом доступе и доступны после регистрации в приложении по прямой ссылке.

# Этапы работы над проектом
* Определение темы
* Сбор датасета
* Обработка датасета
* Аугментация данных
* Подбор модели
* Определение наиболее эффективной модели.

В качестве метрики была использована точность, по аналогии с схожими исследованиями, проводимыми в Wang, Y., & Kosinski, M. (2018). Deep neural networks are more accurate than humans at detecting sexual orientation from facial images. Journal of Personality and Social Psychology, 114(2), 246–257. https://doi.org/10.1037/pspa0000098



In [ ]:
DATA_PATH = '../input/tinder-faces/data'
PATH = "../working" # рабочая директория

In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor
!pip install -q efficientnet

In [ ]:
# Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau 
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
import tensorflow.keras.models as Model
import tensorflow.keras.layers as Layer
from tensorflow.keras.applications.xception import Xception as xcp
import PIL
from PIL import ImageOps, ImageFilter
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A
import efficientnet.keras as efn 
from keras import backend as K
from keras.applications.densenet import DenseNet169
from sklearn.metrics import accuracy_score

# Парсер сайтов знакомств

In [ ]:
# from selenium import webdriver
# from time import sleep
# import urllib
# import re
# import random
# import time
# import uuid
# from tqdm import tqdm

# # Заходим на сайт и сами ручками логинимся, двухфакторная авторизация с капчей мне не поддалась.
# driver.get('https://badoo.com')
# # Сохраняем фото с уникальным именем
# def save_photo():
#     img = driver.find_element_by_xpath('/html/body/div[2]/div[1]/main/div[1]/div/div[1]/section/div/div[1]/div/div[1]/img')
#     link = img.get_attribute('src')
#     urllib.request.urlretrieve(link, "./1/"+str(uuid.uuid4())+".png")
# # Функции для свайпов влево/вправо и закрытием окошка с метчем
# def swipe_right():
#     driver.find_element_by_xpath('/html/body/div[2]/div[1]/main/div[1]/div/div[1]/section/div/div[2]/div/div[2]/div[1]/div[1]').click()
# def swipe_left():
#     driver.find_element_by_xpath('/html/body/div[2]/div[1]/main/div[1]/div/div[1]/section/div/div[2]/div/div[2]/div[2]/div[1]').click()
# def close_match():
#     driver.find_element_by_xpath('/html/body/aside/section/div[1]/div/div[2]/i').click()
# # Имитируем пользователя. Случайно свайпаем влево/вправо и сохраняем фотографии.
# for i in tqdm(range(3000)):
#     if random.random()>0.7:
#         try:
#             swipe_right()
#             time.sleep((0.1+random.random()))
#             save_photo()
#         except Exception:
#             time.sleep(int(10*(1+random.random())))
#             close_match()
#     else:
#         try:
#             swipe_left()
#             time.sleep(0.1+random.random())
#             save_photo()
#         except Exception:
#             time.sleep(int(10*(1+random.random())))
#             close_match()
# # Закрываем браузер
# driver.quit()

# Скрипт по выделению лица на фотографии

In [ ]:

# import face_recognition
# import numpy as np
# from PIL import Image
# import os
# from uuid import uuid4
# import tqdm
# for image in os.listdir('./0'):
#     try:
#         face = face_recognition.load_image_file('./0/'+image)
#         face_locations = face_recognition.face_locations(face)
#         for face_location in face_locations:
#             top, right, bottom, left = face_location
#             try:
#                 face_image = face[top-25:bottom+25, left-25:right+25]
#                 pil_image = Image.fromarray(face_image)
#             except ValueError:
#                 face_image = face[top:bottom, left:right]
#                 pil_image = Image.fromarray(face_image)
#             pil_image.save('./0_faces/'+str(uuid4())+'.png')
#     except Exception:
#         continue

# Ресайзинг фотографий до 512х512
# for image in os.listdir('./1_faces/'):
#     pic = Image.open('./1_faces/'+image)
#     new_image = pic.resize((512, 512))
#     new_image.save('./1_faces_512/'+'512_'+image)
# for image in os.listdir('./0_faces/'):
#     pic = Image.open('./0_faces/'+image)
#     new_image = pic.resize((512, 512))
#     new_image.save('./0_faces_512/'+'512_'+image)

In [ ]:
EPOCHS               = 100  # эпох на обучение
BATCH_SIZE           = 16 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-5
VAL_SPLIT            = 0.20 # сколько данных выделяем на тест = 20%

CLASS_NUM            = 2  # количество классов в нашей задаче
IMG_SIZE             = 512 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
RANDOM_SEED          = 42 # для воспроизводимости

In [ ]:
os.listdir('../input/tinder-faces/data')

In [ ]:
# Загрузка датафрейма
df_0 = pd.DataFrame({'id':os.listdir(DATA_PATH + '/0_faces_512'), 'class':0}, columns = ['id','class'])
df_1 = pd.DataFrame({'id':os.listdir(DATA_PATH+'/1_faces_512'), 'class':1}, columns = ['id','class'])
df = df_0.append(df_1)

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = df.sample(n=9)
random_image_paths = random_image['id'].values
random_image_cat = random_image['class'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_PATH+f'/{random_image_cat[index]}_faces_512/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
# Аугментация данных

transform = A.Compose([
#         A.Transpose(),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=30, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
            A.IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ]) 


In [ ]:
train_gen = ImageDataAugmentor(rescale=1./255,
                        augment=transform, 
                        seed=RANDOM_SEED,
                        validation_split=VAL_SPLIT
                       )


In [ ]:
# Завернем наши данные в генератор:

train_generator = train_gen.flow_from_directory(
    DATA_PATH,      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_gen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

In [ ]:
x,y = train_generator.next()
print('Пример картинок из test_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
plt.show()

In [ ]:
# В качестве базовой модели берем Xception, замораживаем все слои, кроме головы и надстраиваем простую голову 
def baseline_model():
    base_model = xcp(weights='imagenet', include_top=False, input_shape = input_shape)
    base_model.trainable = True
    model=Model.Sequential()
    model.add(base_model)
    model.add(Layer.GlobalAveragePooling2D())
    model.add(Layer.Dense(256, 
                          activation='relu'))
    model.add(Layer.Dense(CLASS_NUM, activation='softmax'))
    return model

In [ ]:
# Определим callbacks для сохранения моделей в процессе обучения
def callbacks(filename):
    checkpoint = ModelCheckpoint(filename + '.hdf5', 
                             monitor = ['val_accuracy'],
                             verbose = 1,
                             mode = 'max')
    earlystop = EarlyStopping(monitor = 'val_accuracy',
                              min_delta = 0.001,
                              patience = 3,
                              restore_best_weights = True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.25,
                                  patience=2,
                                  min_lr=0.0000001,
                                  verbose=1,
                                  mode='auto')
    callbacks_list = [checkpoint, earlystop, reduce_lr]
    return callbacks_list

In [ ]:
def compile_history(model, model_name):
    model.compile(loss="categorical_crossentropy", 
            optimizer=optimizers.Adam(learning_rate=LR), 
            metrics=["accuracy"])
    history_model = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks(model_name)
    )
    return history_model

In [ ]:
# Функция для отбражения графиков перфоманса модели

def show_graphs(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
baseline_model = baseline_model()

In [ ]:
baseline_model.summary()

In [ ]:
history_baseline_model = compile_history(baseline_model,'baseline_model')

In [ ]:
show_graphs(history_baseline_model)

In [ ]:
# В качестве альтернативной модели берем Xception, замораживаем все слои, кроме головы и надстраиваем простую голову 
def model_efn():
    K.clear_session()
    base_model = efn.EfficientNetB5(weights='imagenet', 
                                include_top=False, 
                                input_shape = input_shape)
    base_model.trainable = True
    model=Model.Sequential()
    model.add(base_model)
    model.add(Layer.GlobalAveragePooling2D())
    model.add(Layer.Dense(256, 
                          activation='relu'))
    model.add(Layer.Dense(CLASS_NUM, activation='softmax'))
    return model


In [ ]:
model_efn = model_efn()

In [ ]:
model_efn.summary()

In [ ]:
# EFN модель выдает ошибку OutOfMemory с батчем в 16 и размером в 512 пикселей, для ее работы уменьшаем размер до 256

IMG_SIZE             = 256 # какого размера подаем изображения в сеть
# Завернем наши данные в генератор:

train_generator = train_gen.flow_from_directory(
    DATA_PATH,      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_gen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

history_model_efn = compile_history(model_efn,'model_efn')

In [ ]:
show_graphs(history_model_efn)

In [ ]:
# В качестве дополнительной модели берем DenseNet, замораживаем все слои, кроме головы и надстраиваем простую голову 
def model_densenet():
    base_model = DenseNet169(weights='imagenet', include_top=False, input_shape = input_shape)
    base_model.trainable = True
    model=Model.Sequential()
    model.add(base_model)
    model.add(Layer.GlobalAveragePooling2D())
    model.add(Layer.Dense(256, 
                          activation='relu'))
    model.add(Layer.Dense(CLASS_NUM, activation='softmax'))
    return model

In [ ]:
model_densenet = model_densenet()

In [ ]:
model_densenet.summary()

In [ ]:
IMG_SIZE             = 512 # Для densenet вернем все к начальным условиям
# Завернем наши данные в генератор:

train_generator = train_gen.flow_from_directory(
    DATA_PATH,      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_gen.flow_from_directory(
    DATA_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

history_model_densenet = compile_history(model_densenet,'model_densenet')

In [ ]:
# Вспомогательная функция для получения классов предсказаний
def get_prediction(model,test_sub_generator, tta=0):
            if tta == 0:
                preds = model.predict(test_sub_generator, 
                                          steps=len(test_sub_generator), 
                                          verbose=1)
            else:
                predictions_tta = []
                for i in range(tta):
                    preds = model.predict(test_sub_generator, verbose=1) 
                    predictions_tta.append(preds)
                preds = np.mean(predictions_tta, axis=0) 
            return preds

## Ансамблируем модели для финального предсказания

In [ ]:
# Выделяем предсказания по всем моделям
preds_base = get_prediction(baseline_model,test_generator, tta=5)
preds_efn = get_prediction(model_efn,test_generator, tta=5)
preds_densenet = get_prediction(model_densenet,test_generator, tta=5)

In [ ]:
# Делаем ансамбль по средним вероятностям
predictions = []
predictions.append(preds_base)
predictions.append(preds_efn)
predictions.append(preds_densenet)
preds = np.mean(predictions, axis=0)
preds_label = np.argmax(preds, axis=-1) 

In [ ]:
# Финальная точность с учетом TTA и ансамбля из трех моделей
accuracy_score(test_generator.classes, preds_label)

# Выводы

Данная работа демонстрирует навыки по сбору, обработке и работе с данными, полученными из любых источников.  
К сожалению, точность предсказания оказалась меньше, чем в исследуемой статье, однако больше 50%, значит есть некоторые признаки, позволяющие определить человека к одному или другому классу по фотографии.  
Возможно, стоило использовать другие нейросети, но это уже находится за рамками дипломного проекта.